# MOwNiT - Generatory liczb losowych

## Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, *logarytmem dyskretnym* $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

### Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest pierwiastkiem pierwotnym $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest generatorem grupy multiplikatywnej rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$

## Algorytm wymiany klucza Diffiego-Hellmana

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$ losowa, z rozkładu jednostajnego
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$ losowa, z rozkładu jednostajnego
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 1. (2 pkt)

Zaimplementuj wybrany algorytm generowania liczb losowych z rozkładem jednostajnym w pierścieniu $\mathbb{Z}_p$.

In [2]:
class LCG:
    def __init__(self, seed = 1234):
        self.mod = 1 << 32
        self.multiplier = 1664525 % self.mod
        self.increment = 907633385 % self.mod
        self.state = seed

    def rand(self):
        self.state = (self.state * self.multiplier + self.increment) % self.mod
        return self.state / self.mod

    def uniform(self, low = 0.0, high = 1.0):
        return low + self.rand() * (high - low)

    def rand_int(self, low = 0, high = 1):
        return int(self.uniform(low, high))

## Zadanie 2. (2 pkt)

Wykorzystaj algorytm z zadania pierwszego do implementacji wymiany klucza Diffie-Hellman.

In [12]:
class Endpoint:
    def __init__(self, p: int, q: int, rng: LCG):
        self.p = p
        self.q = q
        self.rng = rng
        self.private_key = self.rng.rand_int(0, self.p)

    def create_public_key(self):
        self.private_key = self.rng.rand_int(0, self.p)
        return pow(self.q, self.private_key, mod = self.p)

    def compute_shared_key(self, other_endpoint_key):
        return pow(other_endpoint_key, self.private_key, mod = self.p)

In [14]:
p = 7
q = 3
rng = LCG()

alice = Endpoint(p, q, rng)
bob = Endpoint(p, q, rng)

alice_public_key = alice.create_public_key()
bob_public_key = bob.create_public_key()

print(f"alice:\npublic_key = {alice_public_key}, private_key = {alice.private_key}")
print(f"bob:\npublic_key = {bob_public_key}, private_key = {bob.private_key}")
print(f"alice:\ncomputed shared key = {alice.compute_shared_key(bob_public_key)}")
print(f"bob:\ncomputed shared key = {bob.compute_shared_key(alice_public_key)}")

alice:
public_key = 1, private_key = 0
bob:
public_key = 5, private_key = 5
alice:
computed shared key = 1
bob:
computed shared key = 1


## Zadanie 3. (1 pkt)
Na swoją własną implementację przeprowadź atak metodą brute force. Jeżeli zakończy się sukcesem - zmień parametry kryptosystemu tak, by atak brute force nie przyniósł rezultatów w rozsądnym czasie (rozsądny czas = 10 minut).

In [15]:
def discrete_logarithm(x, base, mod):
    for k in range(mod):
        if pow(base, k, mod = mod) == x:
            return k

    raise Exception(f"Discreate logarithm log{base}({x}) does not exist!!!")

In [22]:
class BreakEncryption:
    def __init__(self, p, q, public_key_alice, public_key_bob):
        self.p = p
        self.q = q
        self.public_key_alice = public_key_alice
        self.public_key_bob = public_key_bob

    def break_private_keys(self):
        private_key_alice = discrete_logarithm(self.public_key_alice, self.q, self.p)
        private_key_bob = discrete_logarithm(self.public_key_bob, self.q, self.p)

        return private_key_alice, private_key_bob

    def break_shared_key(self):
        private_key_bob = discrete_logarithm(self.public_key_bob, self.q, self.p)
        return pow(alice_public_key, private_key_bob)

#### Próba złamania kluczy prywatnych dla małej liczby pierwszej

In [23]:
break_encryption = BreakEncryption(p, q, alice_public_key, bob_public_key)
break_encryption.break_private_keys()
break_encryption.break_shared_key()

1

#### Próba złamania kluczy prywatnych dla dużej liczby pierwszej

In [24]:
from sympy.ntheory import primitive_root

In [36]:
p = 479001599
q = primitive_root(p)
rng = LCG()

alice = Endpoint(p, q, rng)
bob = Endpoint(p, q, rng)

alice_public_key = alice.create_public_key()
bob_public_key = bob.create_public_key()

print(f"alice:\npublic_key = {alice_public_key}, private_key = {alice.private_key}")
print(f"bob:\npublic_key = {bob_public_key}, private_key = {bob.private_key}")
print(f"alice:\ncomputed shared key = {alice.compute_shared_key(bob_public_key)}")
print(f"bob:\ncomputed shared key = {bob.compute_shared_key(alice_public_key)}")

alice:
public_key = 343065323, private_key = 33506462
bob:
public_key = 228674309, private_key = 373329534
alice:
computed shared key = 317280734
bob:
computed shared key = 317280734


In [37]:
break_encryption = BreakEncryption(p, q, alice_public_key, bob_public_key)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import time

executor = ThreadPoolExecutor(max_workers=1)
future = executor.submit(break_encryption.break_private_keys)

timeout = 600

try:
    start = time.time()
    alice_private_key, bob_private_key = future.result(timeout=timeout)
    end = time.time()

    time_elapsed = end - start

    print(f"alice private key: {alice_private_key}")
    print(f"bob private key: {bob_private_key}")
    print(f"attack time: {time_elapsed}")
except TimeoutError:
    print(f"couldn't break private keys in {timeout} [s]!")

ouldn't break private keys in 600 [s]!
